In [2]:
!pip install --upgrade "jax[cuda12_pip]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install flax

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.6/226.6 kB 948.7 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 2.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.8/239.8 kB 984.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.4/322.4 kB 2.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 3.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 2.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 2.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.6/89.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 2.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# The paper https://arxiv.org/abs/2303.06349 cals the code they provide a "simplified" implmementation - what did they leave out?



In [448]:
#Here is the output without line numbers:

#the help of Lemma 3.2, with phase potentially restricted to a thin slice (see §3.4).
import jax
from jax import random
from flax import linen as nn
import jax.numpy as jnp
import numpy as np
from flax.linen import initializers
from flax.training import train_state
import optax
from flax.training import train_state
from flax.training.common_utils import shard, get_metrics
parallel_scan = jax.lax.associative_scan










class LRU(nn.Module):
    """Linear Recurrent Unit (LRU) layer"""
    state_dim:int
    embed_dim:int
    r_min: float = 0.5
    r_max: float = 0.99
    max_phase: float = 6.28

    def setup(self):
        self.B_re = self.param('B_re', initializers.glorot_normal(), (self.state_dim, self.embed_dim))
        self.B_im = self.param('B_im', initializers.glorot_normal(), (self.state_dim, self.embed_dim))
        self.C_re = self.param('C_re', initializers.glorot_normal(), (self.embed_dim, self.state_dim))
        self.C_im = self.param('C_im', initializers.glorot_normal(), (self.embed_dim, self.state_dim))
        self.D = self.param('D', initializers.normal(), (self.embed_dim,))
        
        u1 = np.random.uniform(size=(self.state_dim,))
        u2 = np.random.uniform(size=(self.state_dim,))
        nu_log = np.log(-0.5*np.log(u1*(self.r_max**2-self.r_min**2) + self.r_min**2))
        theta_log = np.log(self.max_phase*u2)
        
        diag_lambda = np.exp(-jnp.exp(nu_log) + 1j*jnp.exp(theta_log))
        gamma_log = np.log(jnp.sqrt(1-jnp.abs(diag_lambda)**2))

        # Initialize the parameters here
        self.nu_log = self.param('nu_log', lambda rng, shape: nu_log, ())
        self.theta_log = self.param('theta_log', lambda rng, shape: theta_log, ())
        self.gamma_log = self.param('gamma_log', lambda rng, shape: gamma_log, ())

    def __call__(self, input_sequence):
        """Forward pass of the LRU layer. Output y and input_sequence are of shape (L, H)."""

        # Materializing the diagonal of Lambda and projections
        Lambda = jnp.exp(-jnp.exp(self.nu_log) + 1j*jnp.exp(self.theta_log))
        B_norm = (self.B_re + 1j*self.B_im) * jnp.expand_dims(jnp.exp(self.gamma_log), axis=-1)
        C = self.C_re + 1j*self.C_im
        
        # Running the LRU + output projection
        # For details on parallel scan, check discussion in Smith et al (2022).
        if True:
            Lambda_elements = jnp.repeat(Lambda[None, None, :], input_sequence.shape[0], axis=0)
            Lambda_elements = jnp.repeat(Lambda_elements, input_sequence.shape[1], axis=1)

            Bu_elements = jax.vmap(jax.vmap(lambda u: B_norm @ u))(input_sequence)

            

        else:
            Lambda_elements = jnp.repeat(Lambda[None, ...], input_sequence.shape[0], axis=0)
            print('lamba elements',Lambda_elements.shape)

            Bu_elements = jax.vmap(lambda u: B_norm @ u)(input_sequence)
            print('Bu_elements',Bu_elements.shape)

        elements = (Lambda_elements, Bu_elements)
        _, inner_states = parallel_scan(self.binary_operator_diag, elements, axis=1) # all x_k
        #print(Bu_elements)
        #print(inner_states.mean(axis=-1))
        #y = jax.vmap(jax.vmap(lambda x, u: (C @ x).real + self.D * u))(inner_states, input_sequence)
        y = jax.vmap(jax.vmap(lambda x, u: (C @ x).real + self.D * u))(inner_states, input_sequence)

        return y
    
    def binary_operator_diag(self, element_i, element_j):

        # Binary operator for parallel scan of linear recurrence.
        a_i, bu_i = element_i
        a_j, bu_j = element_j

        return a_j * a_i, a_j * bu_i + bu_j
    


# generate random vector
key1 = random.PRNGKey(0)
x1 = jnp.asarray([[[0.1,0.5], [0.2,0.2], [0.3,0.3]],[[0.9,0.9], [0.8,0.8], [0.7,0.7]]]) #jax.random.normal(key1, shape=(1,3,64),)
x2 = jnp.asarray([[[0.1,0.1], [0.2,0.2], [0.3,0.3]],[[0.9,0.9], [0.8,0.8], [0.7,0.7]]]) #jax.random.normal(key1, shape=(1,3,64),)

key2 = jax.random.PRNGKey(0)  # Set a random key for reproducibility
lru = LRU(embed_dim=2, state_dim=2)
params = lru.init(key2, x1)

print('\n\n')
y = lru.apply(params, x2)

In [449]:
class FFW(nn.Module):
    embed_dim: int
    FFW_dim: int

    def setup(self):
        self.up = nn.Dense(self.FFW_dim)
        self.down = nn.Dense(self.embed_dim)
        
    def __call__(self, x):
        x = self.up(x)
        x = nn.activation.silu(x)
        x = self.down(x)
        return x
    

# generate random vector
key1 = random.PRNGKey(0)
x1 = jnp.asarray([[[10,0], [20,25], [2,37]],[[90,90], [80,80], [70,70]]]) #jax.random.normal(key1, shape=(1,3,64),)
x2 = jnp.asarray([[[10,10], [20,20], [30,30]],[[90,90], [80,80], [70,70]]]) #jax.random.normal(key1, shape=(1,3,64),)

key2 = jax.random.PRNGKey(0)  # Set a random key for reproducibility
lru_block = FFW(embed_dim=2, FFW_dim=8)
params = lru_block.init(key2, x1)
print(x1.shape)
y = lru_block.apply(params, x1)
print(y)

y = lru_block.apply(params, x2)
print(y)

(2, 3, 2)
[[[  0.26570448  -0.12417762]
  [  9.270536    -3.246264  ]
  [ 26.527102    -0.87200737]]

 [[ 34.693897   -11.8206415 ]
  [ 30.840815   -10.508144  ]
  [ 26.988638    -9.196105  ]]]
[[[  3.7415366  -1.3212094]
  [  7.7269716  -2.6466095]
  [ 11.5918255  -3.9552112]]

 [[ 34.693897  -11.8206415]
  [ 30.840815  -10.508144 ]
  [ 26.988638   -9.196105 ]]]


In [450]:
class LRU_block(nn.Module):
    embed_dim: int
    FFW_dim: int
    state_dim: int
    r_min: float = 0.5
    r_max: float = 0.99
    max_phase: float = 6.28

    def setup(self):
        self.ffw = FFW(embed_dim=self.embed_dim, FFW_dim=self.FFW_dim)
        self.lru = LRU(embed_dim=self.embed_dim, state_dim=self.state_dim, r_min=self.r_min, r_max=self.r_max, max_phase=self.max_phase)
        self.norm1 = nn.LayerNorm()
        self.norm2 = nn.LayerNorm()

    def __call__(self, x):
        x = self.norm2(x)
        x = x + self.lru(x)
        
        x = self.norm1(x)
        x = x + self.ffw(x)

        return x
    


# generate random vector
key1 = random.PRNGKey(0)
x1 = jnp.asarray([[[10,0], [20,20], [30,30]],[[90,90], [80,80], [70,70]]]) #jax.random.normal(key1, shape=(1,3,64),)
x2 = jnp.asarray([[[10,10], [20,20], [30,30]],[[90,90], [80,80], [70,70]]]) #jax.random.normal(key1, shape=(1,3,64),)

key2 = jax.random.PRNGKey(0)  # Set a random key for reproducibility
lru_block = LRU_block(embed_dim=2, state_dim=8, FFW_dim=8)
params = lru_block.init(key2, x1)

y = lru_block.apply(params, x2)

In [451]:
class LRU_LLM(nn.Module):
    embed_dim: int
    FFW_dim: int
    state_dim: int
    layers: int    
    vocab_size: int

    def setup(self):
        self.embed = nn.Dense(self.embed_dim)
        self.blocks = [LRU_block(embed_dim=self.embed_dim, FFW_dim=self.FFW_dim, state_dim=self.state_dim)]
        self.final_norm = nn.LayerNorm()
        self.unembed = nn.Dense(self.vocab_size)

    def __call__(self, x):

        x = jax.nn.one_hot(x, num_classes=self.vocab_size)

        # embed tokens
        x = self.embed(x)

        # pass through all LRU blocks
        for block in self.blocks:
            x = block(x)

        # final ln
        x = self.final_norm(x)

        # Get transposed weights
        #weights = jnp.transpose(self.embed['params']['embedding']) 
        #unembed = flax.linen.Linear(weights=weights)
                
        # softmax projection
        return self.unembed(x)


In [321]:
# load dataset
vocab = list(set(open('/media/user/Z/PythonQA.txt').read()))
vocab.insert(0, '</s>')
vocab.insert(0, '<s>')
vocab.insert(0, '<unk>')

dataset_samples = open('/media/user/Z/PythonQA.txt').read().split('<|endoftext|>')

print(len(dataset_samples))
char2idx = {ch: i for i, ch in enumerate(vocab)}
idx2char = {i: ch for i, ch in enumerate(vocab)}

vocab_size = 256 #len(vocab)

607283


In [452]:
ctx_size = 512
embed_dim = 256
FFW_dim = 512
state_dim = 128
layers = 2
batch_size = 16


# generate random vector
key1 = random.PRNGKey(0)
x = jax.random.randint(key1, shape=(8,512), minval=0, maxval=10, dtype=jnp.uint32)

key2 = jax.random.PRNGKey(0)  # Set a random key for reproducibility
lru_LLM = LRU_LLM(embed_dim=embed_dim, FFW_dim=FFW_dim, state_dim=state_dim, layers=layers, vocab_size=vocab_size)
params = lru_LLM.init(key2, x)


In [453]:
# Character tokenizer

class SimpleDataLoader:
    def __init__(self, dataset_samples, char2idx, batch_size):
        self.dataset_samples = dataset_samples
        self.char2idx = char2idx
        self.idx2char = idx2char
        self.batch_size = batch_size
        self.vocab_size = len(char2idx)

    def __len__(self):
        return len(self.dataset_samples)

    def get_batch(self, index):
        batch_samples = self.dataset_samples[index : index + self.batch_size]
        batch_input = []
        batch_target = []
        for sample in batch_samples:
            input_ids = self.tokenize(sample)
            batch_input.append(input_ids[1:])
            batch_target.append(input_ids[:-1])
        return jnp.array(batch_input), jnp.array(batch_target)

    def tokenize(self, text, max_length):
        list = [char2idx[ch] for ch in text]
        list.insert(0,1)
        list.append(2)
        list += [2 for i in range(max_length-len(list))]
        list = list[:max_length]
        return list
    
    def detokenize(self, ids):
        return "".join([self.idx2char[i] for i in ids])

data_loader = SimpleDataLoader(dataset_samples, char2idx, batch_size=batch_size)


In [454]:
import jax
import jax.numpy as jnp
from flax import linen as nn

def generate(model, seed, gen_length):
    """
    Generates text using the LRU_LLM language model.
    
    Args:
        model: LRU_LLM model instance.
        seed: Initial seed text to start generation.
        max_length: Maximum length of the generated text.
    
    Returns:
        Generated text as a string.
    """
    current_length = len(seed)
    generated_text = seed

    while current_length < gen_length:
        # Convert the generated text to token IDs
        input_ids = jnp.array([data_loader.tokenize(generated_text, current_length)])

        # Perform a forward pass through the model
        logits = model.apply(params, input_ids)
        print(logits.shape)

        # Sample the next token from the logits
        #next_token_id = jax.random.categorical(logits=logits[0,-1], key=key2).item()
        next_token_id = np.argmax(logits[0,-1][0:217]).item()

        # Convert the next token ID to a character
        next_token = data_loader.detokenize([next_token_id])

        # Append the next token to the generated text
        generated_text += next_token

        current_length += 1

    print(logits[0].mean(axis=-1))

    return generated_text

print(generate(lru_LLM, 'hi', gen_length=32))

(1, 2, 256)
(1, 3, 256)
(1, 4, 256)
(1, 5, 256)
(1, 6, 256)
(1, 7, 256)
(1, 8, 256)
(1, 9, 256)
(1, 10, 256)
(1, 11, 256)
(1, 12, 256)
(1, 13, 256)
(1, 14, 256)
(1, 15, 256)
(1, 16, 256)
(1, 17, 256)
(1, 18, 256)
(1, 19, 256)
(1, 20, 256)
(1, 21, 256)
(1, 22, 256)
(1, 23, 256)
(1, 24, 256)
(1, 25, 256)
(1, 26, 256)
(1, 27, 256)
(1, 28, 256)
(1, 29, 256)
(1, 30, 256)
(1, 31, 256)
[ 0.04205338 -0.00910356  0.03542134  0.10676627  0.05018805  0.03164284
  0.02245259  0.00570123  0.08475644 -0.0323355   0.0337486  -0.02981921
 -0.13637358 -0.06288107  0.03588936  0.05091178 -0.0609285  -0.09887794
  0.02181756 -0.00181809 -0.05359057  0.09079576  0.11184372  0.00979271
 -0.03580133  0.00402478 -0.00833572 -0.07054727 -0.03173715  0.03292423
  0.07681849]
hi9Ò=±/$Y¤^¡ÃÕRîÆ®.
»[DÕ8Z!¢ÌîÌ


In [344]:
input_ids = jnp.asarray([[1,2,3,4,5,6,7,8,9,10],[11,12,13,14,15,12,17,18,19,20]])
logits = lru_LLM.apply(params, input_ids)
print(logits.mean(axis=(2)))

[[ 0.04577552 -0.01078083  0.08290634 -0.01238275  0.14061648 -0.06481279
  -0.09183305  0.01484728  0.051723   -0.02798086]
 [-0.03113073  0.03057463  0.04242035  0.11461998 -0.03873058  0.01570053
  -0.03451788  0.05533995  0.02322705  0.12438513]]


In [347]:
input_ids = jnp.asarray([[1,2,3,4,5,6,7,8,9,10],[10003,12,13,14,15,12,17,18,19,20]])
logits = lru_LLM.apply(params, input_ids)
print(logits.mean(axis=(2)))

[[ 0.04577552 -0.01078083  0.08290634 -0.01238275  0.14061648 -0.06481279
  -0.09183305  0.01484728  0.051723   -0.02798086]
 [ 0.04038882  0.03057463  0.04242035  0.11461998 -0.03873058  0.01570053
  -0.03451788  0.05533995  0.02322705  0.12438513]]


In [348]:
input_ids = jnp.asarray([[100,2,3,4,5,6,7,8,9,10],[11,12,13,14,15,12,17,18,19,20]])
logits = lru_LLM.apply(params, input_ids)
print(logits.mean(axis=(2)))

[[-0.0475834  -0.01078083  0.08290634 -0.01238275  0.14061648 -0.06481279
  -0.09183305  0.01484728  0.051723   -0.02798086]
 [-0.082854    0.03057463  0.04242035  0.11461998 -0.03873058  0.01570053
  -0.03451788  0.05533995  0.02322705  0.12438513]]


In [216]:
# Model and optimizer
tx = optax.adam(learning_rate=1e-3)
state = train_state.TrainState.create(apply_fn=lru_LLM.apply, params=lru_LLM.init(jax.random.PRNGKey(0), jnp.ones([1, ctx_size])), tx=tx)

train_steps = len(data_loader) // batch_size  # Number of steps per epoch

# Training loop
for epoch in range(10):

    # Linear warmup and cosine decay
    lr = tx.init_fn(state.step)
    lr = lr * jnp.minimum(1., state.step/1000) * 0.5 * (1. + jnp.cos(jnp.pi * state.step/10000))
    state = tx.update_fn(lr, state.optim_state, state.params)

    for step in range(train_steps):
        xs, ys = data_loader.get_batch(step * batch_size)

        loss = jnp.mean(nn.softmax_cross_entropy(lru_LLM(xs), ys))
        state = state.apply_gradients(grads=jax.grad(loss)(state.params))

# Generate text
print(detokenize(lru_LLM.generate(temperature=1.0, length=1000)))

NameError: name 'CHECKPOINT_PATH' is not defined

In [ ]:
# Standard language model parameters:
# context length (implicit)
# hidden dim, 

In [14]:
y = lru.apply(params, x)